# Beautiful Soup(bs4) を使ってブログの内容をテキストで抜き出してみる
1. リクエスト取得
2. bs4で解析・テキスト取得
3. (改良版) bs4で解析・テキスト取得

In [1]:
# ライブラリインポート
from bs4 import BeautifulSoup
import requests

## リクエスト取得

In [78]:
url = "https://dev.classmethod.jp/cloud/aws/aws-nw-architectures-net320/"
# url = "https://dev.classmethod.jp/cloud/aws/cloudtrail-athena-s3-object-survey/"
response = requests.get(url)
# 確認
response.text[:100]

'\ufeff<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" prefix="og: http://ogp.me/ns#">\n<head>\n<'

## bs4で解析・テキスト取得

In [79]:
soup = BeautifulSoup(response.text, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [80]:
# 単純に get_text()
soup.get_text()[:100]

'\ufeff\n\n\n\n[レポート] AWS ネットワークアーキテクチャ 総まとめ！ #NET320 #reinvent ｜ Developers.IO\n(window.NREUM||(NREUM={})).loa'

In [81]:
# フィルターを行い get_text()
contents = soup.find('div', class_="single_article_contents")
contents.get_text()[:200].split('\n')

['',
 'こちらはラスベガスで開催された AWS re:Invent2019のセッション',
 'The right AWS network architecture for the right reason #NET320',
 'のレポートです。',
 'Transit Gateway/PrivateLink などの新サービス登場や',
 '既存サービスのアップデートとともに、',
 'AWSにおけるネットワーク構成の選択肢は増え続けていま']

## (改良版) bs4で解析・テキスト取得

### ▼ パラグラフのみ抽出

In [82]:
# p 要素の抽出
texts_p = [c.get_text() for c in contents.find_all('p')]
texts_p[:10]

['こちらはラスベガスで開催された AWS re:Invent2019のセッション\nThe right AWS network architecture for the right reason #NET320\nのレポートです。',
 'Transit Gateway/PrivateLink などの新サービス登場や\n既存サービスのアップデートとともに、\nAWSにおけるネットワーク構成の選択肢は増え続けています。',
 '本セッションは、今のAWSにおけるネットワーク構成が網羅されている\n良いセッションでした。',
 '本ブログでは、このセッションで出てきた\nAWSネットワークアーキテクチャ パターンを紹介 していきます。',
 '',
 '項目が多いので以下に目次を作成しています。\n目次のリンクから気になるアーキテクチャを参照ください。',
 '',
 'ネットワークインフラを構築するときに、まず作成するのは VPCです。\nその VPCをどのように構成するかは重要です。',
 '本章は 1 VPCでシステムを構築・運用する構成 について紹介します。',
 '']

In [84]:
import re

# p 要素の抽出
texts_p = [c.get_text() for c in contents.find_all('p')]
# 空白行削除 + 改行コード削除
texts_p = [t.replace('\n','') for t in texts_p if re.match('\S', t)]
texts_p[:10]

['こちらはラスベガスで開催された AWS re:Invent2019のセッションThe right AWS network architecture for the right reason #NET320のレポートです。',
 'Transit Gateway/PrivateLink などの新サービス登場や既存サービスのアップデートとともに、AWSにおけるネットワーク構成の選択肢は増え続けています。',
 '本セッションは、今のAWSにおけるネットワーク構成が網羅されている良いセッションでした。',
 '本ブログでは、このセッションで出てきたAWSネットワークアーキテクチャ パターンを紹介 していきます。',
 '項目が多いので以下に目次を作成しています。目次のリンクから気になるアーキテクチャを参照ください。',
 'ネットワークインフラを構築するときに、まず作成するのは VPCです。その VPCをどのように構成するかは重要です。',
 '本章は 1 VPCでシステムを構築・運用する構成 について紹介します。',
 '一番シンプルな構成です。システム領域の分割は サブネットやルートテーブル、NACLなどを用いて行います。',
 'システムの規模が大きくなって使えるIPレンジが無くなってきた…',
 'その場合は VPCの CIDR拡張 が行えます。']

### ▼ リストアイテムのみ抽出

In [121]:
from bs4.element import Tag, NavigableString

def parse_li(li):
    """
    リストアイテム(li)のテキストを返す
    ※ li内の入れ子リストは除外する ( find_all('li') でそれ単体のリストアイテムが得られるため)
    """
    buffer = []
    for child in li:
        if type(child) == NavigableString:
            buffer.append(child.string)
        elif type(child) == Tag:
            # リスト構造ではない child のみ返り値に含める
            if child.find_all('li') == []:
                buffer.append(child.get_text())
    return ''.join(buffer)

# li 要素の抽出
texts_li = [parse_li(li) for li in contents.find_all('li')]
# 空白行削除 + 改行コード削除
texts_li = [t.replace('\n','') for t in texts_li if re.match('\S', t)]
texts_li[:10]

['フラットネットワーク アーキテクチャ (Single VPC)',
 'シングルアカウント構成',
 'マルチアカウント構成 (Resource Access Manager)',
 '分割ネットワーク アーキテクチャ (Multi VPC)',
 'シングルアカウント構成',
 'マルチアカウント構成',
 'VPC間の接続 (VPC ピアリング)',
 'VPC間の接続 (Transit Gateway)',
 'サイト間VPN',
 'Transit Gateway']